In [3]:
import os
from requests import get
from json import dumps
import json
from urllib.parse import urlencode



In [4]:
pwd = os.getcwd()
filepath = (pwd + f"/data_sets/")

In [5]:
ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"
AREA_TYPE = 'nation'
AREA_NAME = 'england'

In [87]:
filters = [f"areaType={AREA_TYPE}", f"areaName={AREA_NAME}"]

total_case_data_structure = {
    "date" : "date",
    "area" : "areaName",
    "total_cases" : "cumCasesByPublishDate",
    "case_rate_per_100k" : "cumCasesByPublishDateRate",
    "death_28_days": "cumDeaths28DaysByPublishDate",
    "death_rate" : "cumDeath28DaysByPublishDateRate",
    "cases_by_age" : "cumCasesByAge",
    "cum_male_cases" : "maleCases",
    "cum_female_cases" : "femaleCases"    
}
    #"total_deaths" : "cumDeathsByDate",

hospital_data_structure = {

    "date" : "date",
    "area" : "areaName",
    "patients_in_hospital" : "hospitalCases",
    "people_on_vents" : "covidOccupiedMVBeds",
    "total_hospitalisations" : 'cumAdmissions',
    "hos_by_age" : "cumAdmissionsByAge", 

}

demographic_sex_death_data_structure = {

    "date" : "date", 
    "area" : "areaName",
    "female_deaths" : "femaleDeaths",
    "male_deaths" : "maleDeaths"
}

new_case_data_structure = {
    "date" : "data",
    "area" : "areaName",
    "new_cases" : "newCasesByPublishDate",
    "new_change" : "newCasesByPublishDateChange",
    "new_percentage_change" : "newCasesByPublishDatePercentageChange",
    "new_cases_rolling_rate" : "newCasesByPublishDateRollingRate",
    "new_deaths" : "newDeath28DaysByPublishDate",
    "new_d_change" : "newDeath28DaysByPublishDateChange",
    "new_d_percentage_change" : "newDeaths28DaysByPublishDateChangePercentage" ,
    "new_d_rolling_rate" : "newDeaths28DaysByDeathRollingRate",
    "new_d_by_age" : "newDeaths28DaysByDeathDateAgeDemographics"
}

api_params = {
    "filters": str.join(";", filters),
    #"structure" : dumps(structure, separators=(",", ":")),
    #"latestBy" : "cumCasesByPublishDate"
}
formats = {
    "json"
}

data_structures = {"total_case_data" : total_case_data_structure, "hospital_data" : hospital_data_structure, "demographic_sex_death_data" : demographic_sex_death_data_structure, "new_case_data" : new_case_data_structure}

encoded_params = urlencode(api_params)

#print(f"/v1/data?{encoded_params }")

In [104]:
def get_data():
   for key, value in data_structures.items():
      api_params["structure"] = dumps(value, separators=(",", ":"))
      print(api_params["structure"])
      (f"Data being processed for: {key}")
      for fmt in formats:
         api_params["format"] = fmt
         response = get(ENDPOINT, params=api_params, timeout=10)
         print(response.url)
         if response.status_code >= 400:
            raise RuntimeError(f'Request Failed: {response.text}')

         #print(response.url)
         try:
            data = response.content.decode()
         except ValueError:
            pass
      return data

In [85]:
def save_to_file(name='', fmat=''):
    with open(filepath + name + '.' + fmat, 'a') as file:
        json.dump(get_data(), file)

In [105]:
for key in data_structures:

    save_to_file(key, '.json')

{"date":"date","area":"areaName","total_cases":"cumCasesByPublishDate","case_rate_per_100k":"cumCasesByPublishDateRate","death_28_days":"cumDeaths28DaysByPublishDate","death_rate":"cumDeath28DaysByPublishDateRate","cases_by_age":"cumCasesByAge","cum_male_cases":"maleCases","cum_female_cases":"femaleCases"}
https://api.coronavirus.data.gov.uk/v1/data?filters=areaType%3Dnation%3BareaName%3Dengland&structure=%7B%22date%22%3A%22date%22%2C%22area%22%3A%22areaName%22%2C%22total_cases%22%3A%22cumCasesByPublishDate%22%2C%22case_rate_per_100k%22%3A%22cumCasesByPublishDateRate%22%2C%22death_28_days%22%3A%22cumDeaths28DaysByPublishDate%22%2C%22death_rate%22%3A%22cumDeath28DaysByPublishDateRate%22%2C%22cases_by_age%22%3A%22cumCasesByAge%22%2C%22cum_male_cases%22%3A%22maleCases%22%2C%22cum_female_cases%22%3A%22femaleCases%22%7D&format=json


RuntimeError: Request Failed: {"response":"Invalid structure. The structure must be a flat (non-nested) JSON object. Make sure you use double quotation marks in the structure.","status_code":400,"status":"Bad request"}

In [9]:
# _json=json.loads(open(filepath + 'base_total_cases_data.json', 'r').read(), object_pairs_hook=OrderedDict) 
# out=open('base_total_cases_csv', 'w')
# writer = csv.writer(out)               #create a csv.write
# writer.writerow(_json[0].keys())      # header row
# for row in _json:
#     writer.writerow(row.values())